

<center><h1> CSE 4693/6693 Intro to Machine Learning </h1></center>

## Project name: GAN Channel Modeling (Group 4)

<div style="float: left; margin-right: 20px;">

| Name | NetID |
|:-----|:------|
| Joshua Moore | jjm702 |
| Tirian Judy | tkj105 |
| Claire Johnson | kj1289 |
| Aayam Raj Shakya | as5160 |

</div>

In [29]:
# necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import os
import glob
import pandas as pd
import pickle

In [88]:
# Data parsing


# This is the data we want to load
# Distance (dist): Between the UAV and the receiver.
# Altitude (alt): UAV height above ground.
# Frequency (freq): Operating frequencies measured.
# UAV Speed (vel_x, vel_y): UAV mobility, affecting Doppler shifts.
# Environment Type: Use categorical encoding for rural/open field or terrain information.
# LOS/NLOS Classification: A binary or probabilistic input indicating line-of-sight status.

CENTER_FREQ = 3564


file_path = 'dataset/'

# Read all CSV files
arrays = [pd.read_csv(file).rename(columns=lambda x: x.strip()) for file in glob.glob(file_path + "*.csv")]

# Filter the columns in each dataframe, and include only the ones that exist in the dataframe
filtered_arrays = []

# path loss ewma is ... 
# aod_theta, aod_phi: Angles of Departure in the vertical (elevation) and horizontal (azimuth) planes
# aoa_theta, aoa_phi: Angles of Arrival in the vertical (elevation) and horizontal (azimuth) planes
# These angles are critical for beamforming, MIMO systems, and propagation modeling, as they help in 
# determining the direction of signal transmission (AOD) and reception (AOA), improving channel 
# estimation and system optimization.

# AOD (Angle of Departure) and AOA (Angle of Arrival) are calculated based on the relative positions
# of the transmitter (UAV) and receiver (target) in both horizontal (azimuth) and vertical (elevation) planes.
# 
# Horizontal (Azimuth) Angle (phi):
# phi = atan2((y_target - y_uav), (x_target - x_uav))
# 
# Vertical (Elevation) Angle (theta):
# theta = atan2((z_target - z_uav), sqrt((x_target - x_uav)^2 + (y_target - y_uav)^2))
# 
# These angles are essential for beamforming, signal propagation modeling, and estimating the channel conditions.

# for original peaks and peaks we need to process data and only include when it has 4 items in the array 
# 'peaks' represents the indices where the processed signal exhibits local maxima, potentially indicating key events such as 
# moments of high signal strength or important changes in the UAV's movement (e.g., rapid shifts in altitude or speed). 
# 'orig_peaks' shows the indices of maxima in the raw, unprocessed data, which could reflect more noise or less clarity. 
# These peak points are essential for analyzing signal behavior, especially in the context of parameters like distance ('dist'),
# altitude ('alt'), frequency offset should change, and UAV velocity ('vel_x', 'vel_y') this tells us if the UAV is moving, as they might correlate with specific moments 
# when the signal is strongest or when significant changes in the UAV’s position or mobility occur.




columns_to_keep = ['dist', 'avgSnr', 'freq_offset','avgPower', 'avg_pl','avg_pl_ewma', 'aod_theta',	'od_phi','aoa_theta','aoa_phi', 'peaks', 'orig_peaks','speed', 'stage', 'vel_x', 'vel_y','vel_z']  

#process the data
for df in arrays:
    # Keep only the relevant columns that exist in the dataframe
    existing_columns = [col for col in columns_to_keep if col in df.columns.str.strip()]
    filtered_df = df[existing_columns]
    
    if 'stage' in filtered_df.columns:
        filtered_df = filtered_df[filtered_df['stage'] == 'Flight']
    
    # if 'peaks' in filtered_df.columns:


    #     filtered_df['peaks'] = filtered_df['peaks'].apply(lambda x: [i for i in x.split() if i.strip()][1:-1] if isinstance(x, str) else x)
    #     print(filtered_df['peaks'])

    
    
    filtered_arrays.append(filtered_df)

# Print the first 5 rows of the first dataframe (for verification)
#print(filtered_arrays[0].head())

In [ ]:
class Generator(nn.Module):

    def __init__(self, dim):
        super(Generator, self).__init__()

        self.model = nn.Sequential(nn..Linear(dim, 1024 * 8), 
                                   nn.ReLU(), 
                                   nn.Lay)
    
    #Passes data into first layer & executes sequentially based upon params from self.model()
    def forward(self, x):
        return self.model(x) #Need to go read docs pls hold

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3603139287.py, line 7)

In [ ]:
#Useless Data columns
#index, time, lat, lon, detected, center_freq, wav_type, doppler


#Useful Data columns
#dist - distance; H_dist & v_dist still needed probably?
#
#path_loss - 3 values & average - take average?
#Stage == Flight ONLY
#SNR - 3 values & average - take average?
#Can calculate actual frequency center_freq - freq_offset done

#IDK 
#wav_type - seems only to be ZC
#power - has 3 values & then an average
#est_distance -  Distance off what?
#peaks & original peaks? Seems to only vary if 

